In [28]:
import streamlit as st
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import AgentType, create_sql_agent
from langchain.sql_database import SQLDatabase
from langchain.agents.agent_toolkits.sql.toolkit import SQLDatabaseToolkit
from langchain_core.prompts import PromptTemplate
from IPython.display import Markdown as md

In [36]:
# Read the API key
f = open("keys/.gemini_api_key.txt")
key = f.read()

In [37]:
chat_model = ChatGoogleGenerativeAI(google_api_key=key, 
                                   model="gemini-1.5-pro-latest")

In [38]:
# connect to our database
db = SQLDatabase.from_uri("sqlite:///laptops.db")
print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT DISTINCT brand FROM laptops;")

sqlite
['laptops']


"[('HP',), ('Lenovo',), ('ASUS',), ('RedmiBook',), ('acer',), ('Infinix',), ('APPLE',), ('DELL',), ('MSI',), ('SAMSUNG',), ('realme',), ('ALIENWARE',), ('GIGABYTE',), ('Avita',), ('Nokia',), ('Vaio',), ('Mi',), ('LG',), ('Ultimus',)]"

In [26]:
db.run("SELECT DISTINCT storage FROM laptops;")

"[('SSD',), ('HDD',), ('EMMC',)]"

In [27]:
db.run("SELECT brand, max(price) FROM laptops;")

"[('DELL', 1174131)]"

In [40]:
# set up chat template
prompt = PromptTemplate.from_template(
    """You are a helpful AI assistant expert in querying SQL Database,
        The Database has the name laptops
        and has the following columns - brand, description, price, RAM(GB), storage,
        capacity(GB), processor, warranty(years) and display(inch).
        DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.
        if a question does not seem related to the database, the response should be,
        "I may not be able to provide information about this topic"
        Given the following user question, generate corresponding SQL query, and SQL result, 
        and answer the user question. 
        Question: {question}
        Answer: 
        """
)

In [41]:
sql_toolkit = SQLDatabaseToolkit(db=db, llm=chat_model)
sql_toolkit.get_tools()

sqldb_agent = create_sql_agent(
    llm=chat_model,
    toolkit=sql_toolkit,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

In [42]:
response = sqldb_agent.run(prompt.format(
        question="What is the average price of hp core i5 processor laptop?"
  ))
response



> Entering new SQL Agent Executor chain...
Thought: I need to find the average price of HP laptops with Core i5 processors. First, I'll list the available tables to make sure I'm querying the correct one.
Action: sql_db_list_tables
Action Input: laptopsThought: The 'laptops' table is what we need. Now, I'll check the schema to ensure it has the required columns (brand, price, processor).
Action: sql_db_schema
Action Input: laptops 
CREATE TABLE laptops (
	brand TEXT, 
	description TEXT, 
	price INTEGER, 
	"RAM(GB)" INTEGER, 
	storage TEXT, 
	"capacity(GB)" INTEGER, 
	processor TEXT, 
	"warranty(years)" INTEGER, 
	"display(inch)" TEXT
)

/*
3 rows from laptops table:
brand	description	price	RAM(GB)	storage	capacity(GB)	processor	warranty(years)	display(inch)
HP	HP 14s Intel Core i3 11th Gen - (8 GB/256 GB SSD/Windows 11 Home) 14s - dy2507TU Thin and Light Lapt	35490	8	SSD	256	Core i3	1	14
Lenovo	Lenovo V15 G2 Core i3 11th Gen - (8 GB/512 GB SSD/Windows 11 Home) V15 ITL G2 Laptop	33999

Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 4.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 8.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 16.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 32.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..


```sql
SELECT avg(price) FROM laptops WHERE brand = 'HP' AND processor = 'Core i5'
```The query looks correct. Let's execute it to get the average price.
Action: sql_db_query
Action Input: SELECT avg(price) FROM laptops WHERE brand = 'HP' AND processor = 'Core i5'[(73819.0,)]Final Answer: The average price of an HP laptop with a Core i5 processor is 73819.0. 


> Finished chain.


'The average price of an HP laptop with a Core i5 processor is 73819.0.'

In [43]:
md(response)

The average price of an HP laptop with a Core i5 processor is 73819.0.

In [34]:
response = sqldb_agent.run(prompt.format(
        question="what are the available brands?"
  ))
response



> Entering new SQL Agent Executor chain...
Thought: I need to identify the columns present in the laptops database and formulate a SQL query to retrieve distinct brand names.
Action: sql_db_list_tables
Action Input: laptopsAction: sql_db_schema
Action Input: laptops 
CREATE TABLE laptops (
	brand TEXT, 
	description TEXT, 
	price INTEGER, 
	"RAM(GB)" INTEGER, 
	storage TEXT, 
	"capacity(GB)" INTEGER, 
	processor TEXT, 
	"warranty(years)" INTEGER, 
	"display(inch)" TEXT
)

/*
3 rows from laptops table:
brand	description	price	RAM(GB)	storage	capacity(GB)	processor	warranty(years)	display(inch)
HP	HP 14s Intel Core i3 11th Gen - (8 GB/256 GB SSD/Windows 11 Home) 14s - dy2507TU Thin and Light Lapt	35490	8	SSD	256	Core i3	1	14
Lenovo	Lenovo V15 G2 Core i3 11th Gen - (8 GB/512 GB SSD/Windows 11 Home) V15 ITL G2 Laptop	33999	8	SSD	512	Core i3	1	16
HP	HP 15s Intel Core i3 12th Gen - (8 GB/512 GB SSD/Windows 11 Home) 15s-fq5007TU Thin and Light Laptop	45490	8	SSD	512	Core i3	1	16
*/Thought: 

Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..


```sql
SELECT DISTINCT brand FROM laptops;
```

Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 4.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 8.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 16.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 32.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..


Action: sql_db_query
Action Input: SELECT DISTINCT brand FROM laptops;[('HP',), ('Lenovo',), ('ASUS',), ('RedmiBook',), ('acer',), ('Infinix',), ('APPLE',), ('DELL',), ('MSI',), ('SAMSUNG',), ('realme',), ('ALIENWARE',), ('GIGABYTE',), ('Avita',), ('Nokia',), ('Vaio',), ('Mi',), ('LG',), ('Ultimus',)]Final Answer: The available laptop brands are: HP, Lenovo, ASUS, RedmiBook, acer, Infinix, APPLE, DELL, MSI, SAMSUNG, realme, ALIENWARE, GIGABYTE, Avita, Nokia, Vaio, Mi, LG, and Ultimus. 


> Finished chain.


'The available laptop brands are: HP, Lenovo, ASUS, RedmiBook, acer, Infinix, APPLE, DELL, MSI, SAMSUNG, realme, ALIENWARE, GIGABYTE, Avita, Nokia, Vaio, Mi, LG, and Ultimus.'

In [35]:
md(response)

The available laptop brands are: HP, Lenovo, ASUS, RedmiBook, acer, Infinix, APPLE, DELL, MSI, SAMSUNG, realme, ALIENWARE, GIGABYTE, Avita, Nokia, Vaio, Mi, LG, and Ultimus.